In [4]:
from sentence_transformers import SentenceTransformer
import lancedb
from collections import defaultdict
import re
import pandas as pd
from pathlib import Path
import json
from datetime import datetime
from dotenv import load_dotenv
from copy import deepcopy

In [5]:
encoder = SentenceTransformer('nomic-ai/nomic-embed-text-v1.5', device='mps',trust_remote_code=True)

!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>


In [6]:
base_folder = Path('../wonky_data/')
index_folder = base_folder.joinpath('indexes')

In [7]:
save_folder = Path('wonky_data/data/processed_research')
processed_report_files = list(save_folder.rglob('*.json'))

In [8]:
processed_report_files

[PosixPath('wonky_data/data/processed_research/europe_south_america_overlap_topics/the_role_of_cultural_and_migratory_ties_20250415150541.json'),
 PosixPath('wonky_data/data/processed_research/europe_south_america_overlap_topics/the_geopolitics_of_trade_and_investment_european_20250415150234.json'),
 PosixPath('wonky_data/data/processed_research/europe_south_america_overlap_topics/the_evolving_dynamics_of_security_cooperation_countering_20250415150816.json'),
 PosixPath('wonky_data/data/processed_research/europe_south_america_overlap_topics/the_impact_of_european_colonial_legacies_and_20250415150407.json'),
 PosixPath('wonky_data/data/processed_research/europe_south_america_overlap_topics/the_geopolitics_of_environmental_cooperation_addressing_climate_20250415150657.json'),
 PosixPath('wonky_data/data/processed_research/south_america_research_topics/the_geopolitics_of_environmental_issues_deforestation_climate_20250415151425.json'),
 PosixPath('wonky_data/data/processed_research/south_

In [9]:
parsed_reports = list()
for file in processed_report_files:
    with open(file) as json_file:
        parsed_report = json.load(json_file)
        parsed_report['vector'] = parsed_report['report_vector']
        parsed_reports.append(parsed_report)

In [10]:
db = lancedb.connect(index_folder)

In [11]:
tbl = db.create_table("reports", data=parsed_reports, mode='overwrite')

tbl.create_index(metric="cosine")

In [12]:
tbl.create_fts_index(["report",'summary','topic'], use_tantivy=True, replace=True)

In [13]:
_ = tbl.search(encoder.encode('europppean')).distance_type('cosine').limit(1).to_list()
_[0]

{'sections': [{'section_id': 'title',
   'section_idx': 1,
   'section_text': '## The Role of Cultural and Migratory Ties in Shaping European-South American Relations\n',
   'section_title': 'The Role of Cultural and Migratory Ties in Shaping European-South American Relations',
   'section_vector': [0.1838461011648178,
    1.1700996160507202,
    -3.96600079536438,
    -0.24180112779140472,
    0.7549304962158203,
    1.0767124891281128,
    0.15204396843910217,
    -0.1854224056005478,
    0.18667715787887573,
    -0.1550782322883606,
    -0.6362351179122925,
    0.6904297471046448,
    2.105111598968506,
    -0.1857696920633316,
    1.8652043342590332,
    -0.4649380147457123,
    0.3001730740070343,
    0.3588537573814392,
    -0.7818858027458191,
    2.362413167953491,
    -1.3676228523254395,
    0.3958205580711365,
    0.6084296107292175,
    0.634906530380249,
    1.0484269857406616,
    1.2933614253997803,
    0.4343344271183014,
    -0.06993038207292557,
    -1.928347468376159

In [14]:
_[0][('sections')]

[{'section_id': 'title',
  'section_idx': 1,
  'section_text': '## The Role of Cultural and Migratory Ties in Shaping European-South American Relations\n',
  'section_title': 'The Role of Cultural and Migratory Ties in Shaping European-South American Relations',
  'section_vector': [0.1838461011648178,
   1.1700996160507202,
   -3.96600079536438,
   -0.24180112779140472,
   0.7549304962158203,
   1.0767124891281128,
   0.15204396843910217,
   -0.1854224056005478,
   0.18667715787887573,
   -0.1550782322883606,
   -0.6362351179122925,
   0.6904297471046448,
   2.105111598968506,
   -0.1857696920633316,
   1.8652043342590332,
   -0.4649380147457123,
   0.3001730740070343,
   0.3588537573814392,
   -0.7818858027458191,
   2.362413167953491,
   -1.3676228523254395,
   0.3958205580711365,
   0.6084296107292175,
   0.634906530380249,
   1.0484269857406616,
   1.2933614253997803,
   0.4343344271183014,
   -0.06993038207292557,
   -1.9283474683761597,
   -1.864216923713684,
   0.78979861736297

In [39]:
tbl.search('europppean').limit(1).to_list()

[]

In [19]:
parsed_section_reports = list()
for file in processed_report_files:
    with open(file) as json_file:
        parsed_report = json.load(json_file)
        del parsed_report['report_vector']
        del parsed_report['summary_vector']
        sections = deepcopy(parsed_report['sections'])
        del parsed_report['sections']
        for section in sections:
            section_report = deepcopy(parsed_report)
            section_report['section_id'] = section['section_id']
            section_report['section_idx'] = section['section_idx']
            section_report['section_text'] = section['section_text']
            section_report['section_title'] = section['section_title']
            section_report['vector'] = section['section_vector']
            parsed_section_reports.append(section_report)

In [20]:
len(parsed_section_reports)

149

In [21]:
parsed_section_reports[0]

{'report': '## The Role of Cultural and Migratory Ties in Shaping European-South American Relations\n\n### I. Remittances: Economic and Social Impact\n\n#### A. Global Trends and Significance\n\n*   In 2002, global remittances totaled approximately \\$75 billion, with Latin America being the largest recipient region (RS21910(3)).\n*   Between 1996 and 2002, the Western Hemisphere, primarily Latin American countries, experienced a remittance increase of over 200%, rising from \\$11.5 billion to \\$24.4 billion (RS21910(3)).\n*   Remittances are considered the most stable form of international capital flows, unlike private commercial capital flows that fluctuate with economic cycles (RS21910(4)).\n*   Remittances to Mexico are the second-highest source of foreign currency after oil (R41402(5)).\n\n#### B. Impact on Developing Countries\n\n*   Remittances fund consumption of goods and services, potentially lifting families out of poverty. Examples include Ecuador, where remittances increa

In [22]:
tbl = db.create_table("reports_section", data=parsed_section_reports, mode='overwrite')

[2025-04-15T23:32:47Z WARN  lance::dataset::write::insert] No existing dataset at /Users/jameslittiebrant/DataspellProjects/PKMResearcher/../wonky_data/indexes/reports_section.lance, it will be created


In [23]:
tbl.create_fts_index(["section_text",'section_title'], use_tantivy=True, replace=True)